## Tối ưu hóa danh mục đầu tư  - Sử dụng thuật toán di truyền

Người thực hiện: Cao Đức Anh

#### Nhập các gói cần thiết:

In [1]:
import numpy as np
import pandas as pd
from functools import reduce

### 1:
#### Đọc dữ liệu và kết hợp chúng thành một khung dữ liệu.

In [2]:
files=['hdfc.csv','itc.csv','l&t.csv','m&m.csv','sunpha.csv','tcs.csv']
dfs=[]

for file in files:
    temp=pd.read_csv(file)
    temp.columns=['Date',file.replace('.csv','')]
    dfs.append(temp)

stocks = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)
print(stocks.shape)
stocks.head()

(37, 7)


,Date,hdfc,itc,l&t,m&m,sunpha,tcs
0,June 2018,2108.05,266.05,1271.3,896.80,560.55,1847.20
1,May 2018,2136.15,271.60,1367.6,923.50,480.15,1744.80
2,Apr 2018,1944.60,281.45,1400.6,872.65,528.15,1765.70
3,Mar 2018,1891.45,255.90,1311.9,740.20,495.40,1424.65
4,Feb 2018,1883.80,265.10,1319.1,728.75,535.35,1519.13


### 2:
#### Tính lợi nhuận lịch sử trong 3 tháng, 6 tháng, 12 tháng, 24 tháng và 36 tháng cho mỗi cổ phiếu.

Công thức sử dụng đơn giản là sự tăng giá chia cho giá gốc của cổ phiếu.


In [3]:
def hist_return(months):
    ''' Đầu vào: Các tháng dạng danh sách.
        Đầu ra: Lịch sử lợi nhuận dưới dạng DataFrame. '''
    idx=[]
    df=pd.DataFrame()
    for mon in months:
        temp=(stocks.iloc[0,1:] - stocks.iloc[mon,1:])/(stocks.iloc[mon,1:])
        idx.append(str(mon)+'_mon_return')
        df=pd.concat([df, temp.to_frame().T], ignore_index=True)
    df.index=idx
    return df    

In [4]:
hist_stock_returns=hist_return([3,6,12,24,36])
hist_stock_returns

,hdfc,itc,l&t,m&m,sunpha,tcs
3_mon_return,0.114515,0.039664,-0.030947,0.211564,0.13151,0.296599
6_mon_return,0.125163,0.011212,0.011417,0.194062,-0.017957,0.368094
12_mon_return,0.275866,-0.178478,0.129783,0.330899,0.010911,0.562537
24_mon_return,0.792712,0.084237,0.274461,0.255179,-0.265911,0.44833
36_mon_return,0.974847,0.266844,0.069614,0.399938,-0.358785,0.447535


### 3:
Xác định **Gene** : Một phần của tổng số vốn được giao cho một cổ phiếu. Gen là trọng số.

    Gen có thể là một giá trị thập phân từ 0 đến 1.

In [7]:
gene = np.random.rand()
gene

0.1872516161184712

### 4:
Xác định **Nhiễm sắc thể** (mảng 1D): Tập hợp các gen, tức là các phần nhỏ của tổng số vốn được chỉ định cho mỗi cổ phiếu. Thiết lập bộ trọng số.

Mảng 1d của nó là các giá trị phân số của tất cả các cổ phiếu sao cho tổng của mảng sẽ không lớn hơn 1.
Vì chúng ta có 6 cổ phiếu công ty, chúng ta sẽ tạo ra 6 giá trị phân đoạn (gen) cấu thành 1 nhiễm sắc thể.
    
**Tại sao tổng phải bằng 1?** Vì đây là những phần nhỏ của tổng vốn, giả định tổng vốn là 1 đơn vị.
    
**Làm thế nào để đảm bảo tổng = 1?** Chỉ cần tạo 6 số ngẫu nhiên và sau đó tính một hằng số là 1 / [tổng các số ngẫu nhiên]. Cuối cùng nhân từng số ngẫu nhiên với hằng số đó. Tổng sẽ là 1.

In [8]:
def chromosome(n):
    ''' Đầu vào: Số lượng cổ phiếu.
        Đầu ra: Mảng số ngẫu nhiên'''
    ch = np.random.rand(n)
    return ch/sum(ch)

In [9]:
child=chromosome(6)
print(child,sum(child))

[0.25349726 0.15342483 0.12106357 0.14429618 0.22711076 0.10060739] 1.0


### 5:

Khởi tạo **Quẩn thể ban đầu** (Mảng 2D): Một tập hợp các nhiễm sắc thể được tạo ngẫu nhiên

In [10]:
n=6 # Số lượng mã cổ phiếu
pop_size=100 # Dân số ban đầu

population = np.array([chromosome(n) for _ in range(pop_size)])
print(population.shape)
print(population)

(100, 6)
[[4.01242279e-02 1.83504104e-01 1.14225756e-01 1.03878592e-01
  3.71483396e-01 1.86783925e-01]
 [3.14517824e-01 2.96484539e-01 4.52816549e-02 5.54377200e-02
  2.50429967e-01 3.78482948e-02]
 [1.15581918e-01 7.19754273e-02 1.54122675e-01 1.26616995e-01
  1.44511197e-01 3.87191788e-01]
 [2.00127265e-01 2.05955832e-01 8.91204137e-02 1.99976870e-01
  2.35765392e-01 6.90542265e-02]
 [2.04553768e-01 1.85346615e-01 1.81598677e-01 9.84650760e-02
  1.82392866e-01 1.47642997e-01]
 [1.95822929e-01 2.06047430e-01 1.79044181e-02 3.52389428e-01
  1.99036398e-01 2.87993977e-02]
 [7.10666771e-02 1.79373475e-01 1.05688226e-01 1.63190878e-01
  2.66747097e-01 2.13933648e-01]
 [2.32944308e-01 1.88969386e-01 1.19455711e-01 1.74691938e-01
  1.65231293e-01 1.18707364e-01]
 [2.53359110e-01 1.79979370e-01 5.35730268e-02 1.58186465e-01
  1.41178680e-01 2.13723348e-01]
 [7.40067398e-02 2.87970714e-02 1.12966097e-01 4.66094383e-02
  3.51298313e-01 3.86322340e-01]
 [9.83791281e-02 6.85756529e-02 2.0672739

### 6:

**Fitness function** : 
Tỷ lệ Sharpe, S, là một thước đo để định lượng hiệu suất (Thể lực) của danh mục đầu tư và được tính như sau:
                
                S = (µ − r)/σ
    
    Ở đây µ là lợi nhuận của danh mục đầu tư trong một khoảng thời gian cụ thể hoặc lợi tức trung bình của danh mục đầu tư, 
          r là lãi suất phi rủi ro so với cùng kỳ và 
          σ là độ lệch chuẩn của lợi tức trong một khoảng thời gian nhất định hoặc Độ lệch chuẩn của lợi tức danh mục đầu tư.

      
Lợi tức danh mục đầu tư trung bình = Lợi tức trung bình * Phần nhỏ của Tổng vốn (Nhiễm sắc thể).

Tỷ lệ phi rủi ro = 0,0697 (theo google)

Phương sai của lợi tức danh mục đầu tư = (nhiễm sắc thể * Độ lệch chuẩn)**2 + Hiệp phương sai * Trọng lượng tương ứng trong nhiễm sắc thể.

 #### 6.1 
 Tính giá trị trung bình, độ lệch chuẩn và hiệp phương sai của lợi tức cổ phiếu trong quá khứ.

In [11]:
# Chuyển dữ liệu từ kiểu đối tượng sang kiểu số.
print(hist_stock_returns.info())
cols=hist_stock_returns.columns
hist_stock_returns[cols] = hist_stock_returns[cols].apply(pd.to_numeric, errors='coerce')
print(hist_stock_returns.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3_mon_return to 36_mon_return
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hdfc    5 non-null      object
 1   itc     5 non-null      object
 2   l&t     5 non-null      object
 3   m&m     5 non-null      object
 4   sunpha  5 non-null      object
 5   tcs     5 non-null      object
dtypes: object(6)
memory usage: 140.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3_mon_return to 36_mon_return
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hdfc    5 non-null      float64
 1   itc     5 non-null      float64
 2   l&t     5 non-null      float64
 3   m&m     5 non-null      float64
 4   sunpha  5 non-null      float64
 5   tcs     5 non-null      float64
dtypes: float64(6)
memory usage: 260.0+ bytes
None


#### Tính hiệp phương sai của lịch sử lợi nhuận 

In [12]:
cov_hist_return=hist_stock_returns.cov()

print(cov_hist_return)

# Để dễ tính toán, coi hiệp phương sai của cùng một biến bằng không.
for i in range(6):
    cov_hist_return.iloc[i][i]=0
    
cov_hist_return

            hdfc       itc       l&t       m&m    sunpha       tcs
hdfc    0.160272  0.045393  0.027916  0.024127 -0.079078  0.014362
itc     0.045393  0.025467 -0.000718  0.004381 -0.023178 -0.005554
l&t     0.027916 -0.000718  0.014206  0.002510 -0.013841  0.007330
m&m     0.024127  0.004381  0.002510  0.007412 -0.011042  0.005700
sunpha -0.079078 -0.023178 -0.013841 -0.011042  0.041781 -0.007211
tcs     0.014362 -0.005554  0.007330  0.005700 -0.007211  0.009923


,hdfc,itc,l&t,m&m,sunpha,tcs
hdfc,0.000000,0.045393,0.027916,0.024127,-0.079078,0.014362
itc,0.045393,0.000000,-0.000718,0.004381,-0.023178,-0.005554
l&t,0.027916,-0.000718,0.000000,0.002510,-0.013841,0.007330
m&m,0.024127,0.004381,0.002510,0.000000,-0.011042,0.005700
sunpha,-0.079078,-0.023178,-0.013841,-0.011042,0.000000,-0.007211
tcs,0.014362,-0.005554,0.007330,0.005700,-0.007211,0.000000


#### Tính giá trị trung bình của lợi nhuận lịch sử

In [13]:
mean_hist_return=hist_stock_returns.mean()
mean_hist_return

hdfc      0.456621
itc       0.044696
l&t       0.090865
m&m       0.278328
sunpha   -0.100047
tcs       0.424619
dtype: float64

#### Tính toán độ lệch chuẩn của lợi nhuận lịch sử

In [14]:
sd_hist_return=hist_stock_returns.std()
sd_hist_return

hdfc      0.400340
itc       0.159583
l&t       0.119189
m&m       0.086091
sunpha    0.204405
tcs       0.099615
dtype: float64

#### 6.2:
 Tính toán lợi nhuận danh mục đầu tư kỳ vọng và phương sai danh mục đầu tư.

#### Tính toán lợi nhuận kỳ vọng của danh mục đầu tư

In [15]:
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))

In [16]:
mean_portfolio_return(population[0])

0.10796130435880255

#### Tính toán phương sai danh mục đầu tư

In [17]:
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(6):
        for j in range(6):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

In [18]:
var_portfolio_return(population[0])

0.0006384118012693982

#### Yếu tố phi rủi ro

In [19]:
rf= 0.0697

#### Hàm thể lực của một danh mục đầu tư

In [20]:
def fitness_fuction(child):
    ''' Hàm trả về tỷ lệ Sharpe cho một danh mục đầu tư cụ thể.
         Đầu vào: Một nhiễm sắc thể (Mảng 1D)
         Đầu ra: Giá trị Sharpe Ratio (Vô hướng) '''
    return (mean_portfolio_return(child)-rf)/np.sqrt(var_portfolio_return(child))

In [21]:
fitness_fuction(population[7])

1.3096085059228544

### 7:
Chọn **Dân số ưu tú**: Lọc các nhiễm sắc thể ưu tú có lợi nhuận cao nhất, được tính trong chức năng thể dục.

In [23]:
def Select_elite_population(population, frac=0.3):
    ''' Chọn dân số ưu tú từ tổng dân số dựa trên các giá trị thể lực.
         Đầu vào: Toàn bộ dân số.
         Đầu ra: Quần thể ưu tú.'''
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

In [24]:
print(len(Select_elite_population(population, frac=0.3)))
Select_elite_population(population, frac=0.3)

30


[array([0.06342434, 0.2858843 , 0.02972044, 0.02515799, 0.27618801,
        0.31962492]),
 array([0.02485004, 0.06606354, 0.13374255, 0.1829445 , 0.18317564,
        0.40922373]),
 array([0.1007574 , 0.07048682, 0.0737643 , 0.24111433, 0.26724175,
        0.2466354 ]),
 array([0.07106668, 0.17937347, 0.10568823, 0.16319088, 0.2667471 ,
        0.21393365]),
 array([0.15221017, 0.02432027, 0.04740123, 0.16840221, 0.20655765,
        0.40110847]),
 array([0.01713014, 0.08876076, 0.03526185, 0.53866228, 0.05366171,
        0.26652325]),
 array([0.1295624 , 0.05995305, 0.07880163, 0.22365193, 0.26706925,
        0.24096176]),
 array([0.06180811, 0.14357425, 0.06982537, 0.27731635, 0.10669585,
        0.34078008]),
 array([0.02888083, 0.14355099, 0.20397029, 0.19376887, 0.15012781,
        0.27970121]),
 array([0.17873249, 0.00313569, 0.11778031, 0.09858444, 0.36710968,
        0.2346574 ]),
 array([0.11558192, 0.07197543, 0.15412268, 0.12661699, 0.1445112 ,
        0.38719179]),
 array([0.

In [25]:
[fitness_fuction(x) for x in population][:3]

[1.5142909196637127, 0.8079979297203713, 2.580852216433207]

### 8:
**Đột biến**: Một chức năng sẽ thực hiện đột biến trong nhiễm sắc thể. 
            
    Chọn ngẫu nhiên 2 số giữa [0, 5] và các phần tử đó sẽ được đổi chỗ cho nhau.


In [26]:
def mutation(parent):
    ''' Chọn ngẫu nhiên các phần tử của nhiễm sắc thể và hoán đổi vị trí
        Đầu vào: Cha mẹ
        Đầu ra: Con cái '''
    child=parent.copy()
    n=np.random.choice(range(6),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(6),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

In [27]:
mutation(population[1]),population[1]

(array([0.25042997, 0.29648454, 0.04528165, 0.05543772, 0.31451782,
        0.03784829]),
 array([0.31451782, 0.29648454, 0.04528165, 0.05543772, 0.25042997,
        0.03784829]))

### 9:
Phép lai chéo: **Phép lai chéo theo phương pháp Heuristic** hoặc **Phép lai hỗn hợp** 

Đứa con được tạo ra theo phép lai Heuristic:
            
    Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
    Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
        Trong đó β là một số ngẫu nhiên từ 0 đến 1.
Kiểu giao nhau này phù hợp với những bộ gen có giá trị thực.

Đứa con được tạo ra theo phép lai hỗn hợp:

    Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
    Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
        Trong đó α là một số ngẫu nhiên từ 0 đến 1.
Phép lai này thường cho kết quả tốt hơn.

In [30]:
def Heuristic_crossover(parent1,parent2):
    ''' Đứa con được tạo ra theo công thức:
            Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
            Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
                Trong đó β là một số ngẫu nhiên từ 0 đến 1.
        Đầu vào: 2 Cha (Mẹ)
        Đầu ra: 2 Con '''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

In [32]:
for i in population[:30]:
    for j in population[:30]:
        print(Heuristic_crossover(i,j))

(array([0.04012423, 0.1835041 , 0.11422576, 0.10387859, 0.3714834 ,
       0.18678392]), array([0.04012423, 0.1835041 , 0.11422576, 0.10387859, 0.3714834 ,
       0.18678392]))
(array([-0.00342006,  0.16557492,  0.12516669,  0.11156581,  0.3906937 ,
        0.21041894]), array([0.35806211, 0.31441372, 0.03434072, 0.0477505 , 0.23121966,
       0.01421328]))
(array([ 0.17682051, -0.018537  ,  0.1865015 ,  0.14507062, -0.0396908 ,
        0.54983518]), array([-0.02111436,  0.27401653,  0.08184693,  0.08542497,  0.55568539,
        0.02414053]))
(array([-0.00443266,  0.17725185,  0.12121697,  0.0771176 ,  0.40927751,
        0.21956874]), array([0.24468416, 0.21220808, 0.0821292 , 0.22673787, 0.19797128,
       0.03626942]))
(array([-0.10400885,  0.18188903,  0.05516905,  0.10862389,  0.53723342,
        0.22109347]), array([0.34868685, 0.18696169, 0.24065538, 0.09371978, 0.01664284,
       0.11333345]))
(array([0.01955384, 0.18052576, 0.12695141, 0.07104617, 0.39426651,
       0.20765631

(array([ 0.26590579, -0.11692016, -0.01233497,  0.10877578,  0.43705548,
        0.31751809]), array([ 0.17489287,  0.32981968,  0.12804141,  0.23496404,  0.15854515,
       -0.02626314]))
(array([0.0152627 , 0.01768822, 0.30922073, 0.37363238, 0.12446364,
       0.15973231]), array([0.20945892, 0.21545926, 0.07801012, 0.19121103, 0.24138372,
       0.06447695]))
(array([ 0.17136697,  0.12674987, -0.07813188,  0.32192288,  0.37470154,
        0.08339063]), array([ 0.25837442,  0.36636875,  0.42785031, -0.04699588, -0.04561685,
        0.04001925]))
(array([ 0.21234775,  0.26680012, -0.05958036,  0.23039135,  0.25161481,
        0.09842633]), array([0.17307916, 0.07128669, 0.41824591, 0.13265926, 0.20068524,
       0.00404375]))
(array([ 0.19369209, -0.08794454,  0.28406524,  0.01571982,  0.27817822,
        0.31628917]), array([ 0.20292325,  0.33365122,  0.00441976,  0.28003385,  0.21733764,
       -0.03836572]))
(array([0.11647378, 0.3033888 , 0.00696745, 0.21761761, 0.30800688,
     

        0.06037227]))
(array([ 0.50007178, -0.04932408,  0.09434639, -0.0795531 , -0.07000702,
        0.60446603]), array([ 0.07086969,  0.31092703, -0.01348986,  0.52978571,  0.30953096,
       -0.20762354]))
(array([ 0.24897674,  0.19701846, -0.09653218,  0.44086306,  0.17757159,
        0.03210234]), array([-0.02948997,  0.24432021,  0.50298803, -0.02264014,  0.29002326,
        0.01479861]))
(array([ 0.09140522,  0.19393372,  0.00469873,  0.57542431,  0.23631746,
       -0.10177943]), array([ 0.43052992,  0.23327627,  0.04758778, -0.14894168,  0.11523715,
        0.32231056]))
(array([0.09199121, 0.0712101 , 0.36567339, 0.15819788, 0.07591927,
       0.23700815]), array([ 0.22995681,  0.25037417, -0.09642197,  0.41622842,  0.23951021,
       -0.03964764]))
(array([0.16400769, 0.03410663, 0.34035675, 0.21339349, 0.03162308,
       0.21651236]), array([ 0.20590549,  0.26053714, -0.08428386,  0.39643859,  0.2520913 ,
       -0.03068866]))
(array([0.0824358 , 0.17785577, 0.1025513 , 0

(array([ 0.49957506, -0.06123236,  0.01348998,  0.0305713 , -0.06871066,
        0.58630667]), array([ 0.10848779,  0.30575726,  0.16891784,  0.24196382,  0.27442951,
       -0.09955622]))
(array([ 0.40692216,  0.15046137, -0.10424742,  0.26518753,  0.07933395,
        0.20234242]), array([-0.15031402,  0.27379926,  0.61225457, -0.0246621 ,  0.3544558 ,
       -0.0655335 ]))
(array([0.17941859, 0.17047415, 0.1683312 , 0.23248673, 0.17253506,
       0.07675427]), array([ 0.37963793,  0.2396578 , -0.0144934 ,  0.01629841,  0.14521444,
        0.23368483]))
(array([0.09881975, 0.09676587, 0.28506131, 0.23413928, 0.10390896,
       0.18130483]), array([0.26024965, 0.20774036, 0.0857414 , 0.16258953, 0.17771541,
       0.10596365]))
(array([0.16095744, 0.0661989 , 0.26465827, 0.2759078 , 0.06670317,
       0.16557442]), array([0.24607711, 0.21136682, 0.09296592, 0.15622679, 0.18320611,
       0.11015725]))
(array([0.27140023, 0.17968115, 0.04844139, 0.16278128, 0.12169334,
       0.21600261

       0.38632234]))
(array([ 0.05168904, -0.00762802,  0.02710924, -0.11158101,  0.51578815,
        0.5246226 ]), array([0.12069683, 0.10500075, 0.29258424, 0.37755419, 0.00717483,
       0.09698917]))
(array([0.05840527, 0.03265753, 0.09504375, 0.0034534 , 0.40643243,
       0.40400762]), array([0.17155025, 0.00466072, 0.22502014, 0.31642954, 0.00658888,
       0.27575048]))
(array([-0.04548923, -0.01578694,  0.0618575 ,  0.02698568,  0.45868173,
        0.51375126]), array([0.40621024, 0.15274239, 0.25505001, 0.10116426, 0.05276818,
       0.03206493]))
(array([ 0.08223582,  0.00362251,  0.08783437, -0.00671165,  0.37646384,
        0.45655512]), array([0.0303269 , 0.16242335, 0.246365  , 0.32963716, 0.21772001,
       0.01352759]))
(array([0.05001029, 0.03194353, 0.12540314, 0.03262123, 0.35006916,
       0.40995265]), array([0.26466785, 0.00379722, 0.01414898, 0.15775116, 0.36106438,
       0.19857042]))
(array([ 0.09719312,  0.0295504 ,  0.02608855, -0.10273024,  0.45510236,
   

...

(array([0.03156014, 0.09169008, 0.21338926, 0.31070268, 0.25879649,
       0.09386134]), array([0.06222414, 0.29138205, 0.24777099, 0.15997997, 0.18905589,
       0.04958696]))
(array([ 0.30419159, -0.07488026, -0.04649009,  0.12642959,  0.41288261,
        0.27786656]), array([-0.00829189,  0.32764729,  0.31300308,  0.21169995,  0.15191947,
        0.0040221 ]))
(array([ 0.01460617, -0.0440931 ,  0.31708113,  0.42045797,  0.10566579,
        0.18628204]), array([0.06521649, 0.31710811, 0.22095628, 0.13877516, 0.22938303,
       0.02856093]))
(array([0.04565417, 0.24355373, 0.23879209, 0.20092279, 0.21109653,
       0.05998069]), array([0.23918827, 0.2894324 , 0.2617329 , 0.06839392, 0.08718962,
       0.05406289]))
(array([0.00852084, 0.28664927, 0.22929135, 0.20560357, 0.20081296,
       0.06912201]), array([0.23200711, 0.09130505, 0.28018077, 0.15183675, 0.22068854,
       0.02398178]))
(array([ 0.25809709, -0.05012558,  0.18167371,  0.05277786,  0.28364379,
        0.27393313]), ar

       0.00932608]))
(array([-0.15789477,  0.13876476,  0.22615678,  0.42019666,  0.26603756,
        0.10673901]), array([ 0.53133527,  0.1334461 ,  0.20888311, -0.08459566,  0.18481072,
        0.02612046]))
(array([ 0.15198631, -0.1135631 , -0.14964921,  0.22328414,  0.50279644,
        0.38514542]), array([ 0.4235696 ,  0.25546886,  0.39004184, -0.02023625,  0.06500154,
       -0.11384559]))
(array([-0.09002531, -0.01261821,  0.32925186,  0.47774689,  0.10131319,
        0.19433157]), array([ 0.44950418,  0.17477194,  0.18266519, -0.05359541,  0.23673153,
        0.00992257]))
(array([ 0.43505698, -0.009868  ,  0.16928118,  0.04437836,  0.31705436,
        0.04409713]), array([ 0.12944167,  0.43199286,  0.30512345,  0.08985671, -0.01577231,
        0.05935762]))
(array([0.10065087, 0.13052888, 0.30108709, 0.22180692, 0.22138541,
       0.02454083]), array([0.4195333 , 0.13656417, 0.18226466, 0.00055175, 0.20311199,
       0.05797413]))
(array([ 0.11994289, -0.01290918,  0.1913769 ,

       -0.14884507]))
(array([-0.038635  ,  0.09282085,  0.26557116,  0.40567346,  0.40204808,
       -0.12747855]), array([ 0.50159208,  0.04416449,  0.00101682,  0.0037476 , -0.07003196,
        0.51951097]))
(array([ 0.39468858,  0.01693175, -0.01042431,  0.11659812, -0.04097327,
        0.52317912]), array([ 0.31531448,  0.17358584,  0.28718303,  0.04053   ,  0.28942357,
       -0.10603692]))
(array([ 0.43828914, -0.16536471,  0.05821134, -0.01596785,  0.04729055,
        0.63754153]), array([ 0.23703839,  0.53844944,  0.07331464,  0.34661468,  0.02561736,
       -0.22103451]))
(array([0.37511854, 0.05555552, 0.06295212, 0.09784319, 0.04048755,
       0.36804309]), array([0.37511854, 0.05555552, 0.06295212, 0.09784319, 0.04048755,
       0.36804309]))
(array([ 0.69927532, -0.11021991, -0.2373575 ,  0.12736665, -0.1698688 ,
        0.69080424]), array([-0.30049294,  0.40106666,  0.68886105,  0.03631003,  0.4789148 ,
       -0.30465959]))
(array([ 0.41476883, -0.07843466,  0.0860677 

(array([ 0.41021211,  0.21350625, -0.02943236,  0.03675255,  0.13623476,
        0.23272668]), array([-0.0288716 ,  0.25347965,  0.30374141,  0.23170723,  0.22125031,
        0.01869299]))
(array([0.06208385, 0.0663263 , 0.22193028, 0.33724727, 0.30655402,
       0.00585828]), array([0.35186689, 0.23626608, 0.01608767, 0.0484238 , 0.13749277,
       0.20986279]))
(array([ 0.32249274,  0.25672904, -0.0396488 ,  0.08020312,  0.12964169,
        0.25058222]), array([ 0.33850399,  0.0993956 ,  0.2878375 ,  0.05317502,  0.23083927,
       -0.00975137]))
(array([0.33734755, 0.17936422, 0.01955165, 0.00525369, 0.20460979,
       0.25387311]), array([ 0.28897366,  0.35932754,  0.08340431,  0.30164316, -0.01967122,
       -0.01367744]))
(array([ 0.42285019, -0.10574398,  0.09077902,  0.12097542, -0.06862912,
        0.53976847]), array([0.27838057, 0.38246205, 0.00655518, 0.05096096, 0.26163488,
       0.02000635]))
(array([ 0.44882253,  0.21543023, -0.10931263,  0.07744435,  0.105438  ,
      

       0.05711423]))
(array([0.12075581, 0.09901308, 0.26403881, 0.26296079, 0.05621422,
       0.19701729]), array([ 0.41150516,  0.04222686,  0.12300834,  0.23287899,  0.21138263,
       -0.02100198]))
(array([0.21262747, 0.03762046, 0.23759834, 0.27789306, 0.04567813,
       0.18858254]), array([0.01669109, 0.29679923, 0.24049711, 0.17391617, 0.24396671,
       0.0281297 ]))
(array([ 0.04605362,  0.07795798,  0.18690643,  0.33780385,  0.39173799,
       -0.04045987]), array([ 0.21587516,  0.09206818,  0.25489791,  0.23121667, -0.01553143,
        0.22147351]))
(array([0.11174375, 0.07061845, 0.24761457, 0.33427648, 0.03687545,
       0.1988713 ]), array([ 0.39723103,  0.15293996,  0.20436052, -0.0175489 ,  0.25576529,
        0.00725211]))
(array([ 0.06518805, -0.10908477,  0.38461163,  0.27871817,  0.12980189,
        0.25076503]), array([ 0.40911119,  0.51521032, -0.07786928,  0.21152813, -0.01270355,
       -0.0452768 ]))
(array([0.43758528, 0.04528854, 0.00850607, 0.04824988, 0.

In [33]:
def Arithmetic_crossover(parent1,parent2):
    ''' Đứa con được tạo ra theo công thức:
            Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
            Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
                Trong đó α là một số ngẫu nhiên từ 0 đến 1.
        Đầu vào: 2 Cha (Mẹ)
        Đầu ra: 2 Con '''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

In [34]:
Arithmetic_crossover(population[2],population[3])

(array([0.14176333, 0.11346555, 0.13399323, 0.14933457, 0.17277016,
        0.28867316]),
 array([0.17394585, 0.16446571, 0.10924986, 0.17725929, 0.20750643,
        0.16757285]))

### 10:
**Thế hệ tiếp theo**: Thực hiện đột biến, giao phối (trao đổi chéo) dựa trên xác suất và tạo ra thế hệ nhiễm sắc thể mới.

In [35]:
def next_generation(pop_size,elite,crossover=Heuristic_crossover):
    ''' Tạo ra quần thể mới từ quần thể ưu tú với xác suất đột biến là 0,4 và giao chéo là 0,6.
        Trong các giai đoạn cuối cùng, xác suất đột biến giảm xuống 0,1.
        Đầu vào: Quần thể ban đầu
        Output: Quần thể thế hệ tiếp theo.'''
    new_population=[]
    elite_range=range(len(elite))
#     print(elite_range)
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # Trong giai đoạn cuối, tần số đột biến giảm.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
#         print(mutate_or_crossover)
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(6):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if sum((0,chk))>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            new_population.extend([c1,c2])
    return new_population

In [36]:
elite=Select_elite_population(population)
next_generation(100,elite)[:3]

[array([0.06607713, 0.01795359, 0.13336827, 0.29654653, 0.14911456,
        0.33693992]),
 array([0.26562261, 0.12550454, 0.1659715 , 0.29234919, 0.05594963,
        0.09460253]),
 array([0.05594963, 0.12550454, 0.26562261, 0.29234919, 0.1659715 ,
        0.09460253])]

In [37]:
elite=Select_elite_population(population)
next_generation(100,elite,Arithmetic_crossover)[:3]

[array([0.1388669 , 0.07782265, 0.18423322, 0.13883793, 0.19349276,
        0.26674654]),
 array([0.16620835, 0.08406716, 0.16904286, 0.08445872, 0.20823331,
        0.28798959]),
 array([0.1988856 , 0.08342394, 0.13233598, 0.02750945, 0.22669217,
        0.33115285])]

### 11:
**Lặp lại quy trình**: Lặp lại toàn bộ quy trình cho đến khi chúng không thay đổi về lợi nhuận tối đa / rủi ro tối thiểu hoặc đối với số lần lặp cố định.

### Với Heuristic_crossover:

In [38]:
n=6 # Số mã cổ phiếu
pop_size=100 # Số lượng dân số

# Dân số ban đầu
population = np.array([chromosome(n) for _ in range(pop_size)])

# Dân số ưu tú
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns > 0.30 and Expected_risk < 0.0005) or iteration <= 40:
    print('Iteration:',iteration)
    population = next_generation(100,elite)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1


print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

Iteration: 0
Expected returns of 0.14014674006359873 with risk of 0.00022753795657661727

Iteration: 1
Expected returns of 0.1630701045753844 with risk of 0.0006231596450277533

Iteration: 2
Expected returns of 0.17854173218002442 with risk of 0.0006988499623985975

Iteration: 3
Expected returns of 0.21351634473300302 with risk of 0.0014305226225502354

Iteration: 4
Expected returns of 0.19481355409265738 with risk of 0.001224019931776147

Iteration: 5
Expected returns of 0.1886837508596627 with risk of 0.0008581652782976264

Iteration: 6
Expected returns of 0.18664098226486345 with risk of 0.0008143240577356409

Iteration: 7
Expected returns of 0.20480489620325407 with risk of 0.0013192533183282577

Iteration: 8
Expected returns of 0.18664098226486345 with risk of 0.0008143240577356409

Iteration: 9
Expected returns of 0.18029959802553255 with risk of 0.0006830694999496439

Iteration: 10
Expected returns of 0.17774637471453114 with risk of 0.0005857093290873271

Iteration: 11
Expected

[None, None, None, None, None, None]

### Trọng lượng và lợi nhuận tương ứng của chúng:

In [39]:
print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))

Portfolio of stocks after all the iterations:

hdfc : 0.020866757633386027
itc : 0.3028162565216731
l&t : 0.02679192742562279
m&m : 0.003993175586819547
sunpha : 0.2893475627106384
tcs : 0.35618432012186

Expected returns of 0.1489031178443576 with risk of 0.00020918364945660042



Mặc dù phương pháp heuristic hoạt động tốt, nhưng trong 1 số lần thử nghiệm nó mang lại lợi nhuận âm không được mong đợi.

## Phương pháp sử dụng Arithmetic_crossover:

In [40]:
n=6 # Số mã cổ phiếu
pop_size=100 # Số lượng dân số

# Dân số ban đầu
population = np.array([chromosome(n) for _ in range(pop_size)])

# Dân số ưu tú
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns > 0.30 and Expected_risk < 0.0005) or iteration <= 40:
    print('Iteration:',iteration)
    population = next_generation(100,elite,Arithmetic_crossover)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1


print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

Iteration: 0
Expected returns of 0.15748728801367995 with risk of 0.0004265499405916419

Iteration: 1
Expected returns of 0.1581187850866291 with risk of 0.00043200395456817996

Iteration: 2
Expected returns of 0.15748728801367995 with risk of 0.0004265499405916419

Iteration: 3
Expected returns of 0.158001902298611 with risk of 0.00043704247318828056

Iteration: 4
Expected returns of 0.1580459176510951 with risk of 0.0004384197805906015

Iteration: 5
Expected returns of 0.15909714898044106 with risk of 0.000446167376133187

Iteration: 6
Expected returns of 0.15882364306079305 with risk of 0.00043929827368166877

Iteration: 7
Expected returns of 0.1597701822238517 with risk of 0.00044532871112885463

Iteration: 8
Expected returns of 0.15868269687903908 with risk of 0.00043454488377784976

Iteration: 9
Expected returns of 0.158723276790514 with risk of 0.00043499672583776575

Iteration: 10
Expected returns of 0.15947498362260687 with risk of 0.00044231731169973396

Iteration: 11
Expecte

[None, None, None, None, None, None]

In [41]:
print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))

Portfolio of stocks after all the iterations:

hdfc : 0.02341314513540176
itc : 0.2185945307072833
l&t : 0.10574302672242938
m&m : 0.1207001853784263
sunpha : 0.24771609633857686
tcs : 0.2838330157178823

Expected returns of 0.1594016146710573 with risk of 0.00044150054566525175



Phương pháp này thường cho kết quả tốt hơn sau nhiều lần thử nghiệm.